<a href="https://colab.research.google.com/github/mustafabozkaya/DeepLearningBootcamp2022/blob/master/Spectogram_Recognition_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img align="Left" width="100" height="100" src="https://thumbs.dreamstime.com/b/mb-initial-letter-vector-logo-icon-mb-initial-letter-vector-logo-icon-204517753.jpg">](https://github.com/mustafabozkaya)

# Spectogram Recognition with CNN

---
[<img align="Left" width="800" height="300" src="https://www.researchgate.net/publication/319081627/figure/fig1/AS:534034566004736@1504335170521/Spectrogram-of-a-speech-signal-with-breath-sound-marked-as-Breath-whose-bounds-are.png">](#)



**Colab** için kimlik doğrulama adımları:

**Drive yükleme işlemi**

**Drive da dosya konumlandırmayı yapma işlemleri**

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive/')

Mounted at /content/mydrive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#unmount drive
# drive.flush_and_unmount()

In [ ]:
!pwd


/content


In [ ]:
%cd /content

/content


In [ ]:
!pwd

/content


In [ ]:
#Copying current content to new editable directory
#!cp -r "../content/drive/MyDrive/spectrograms/" "/sample_data/"


+
# Kurulum ve Kontroller

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16182412183513684204
 xla_global_id: -1]

In [ ]:
!cat /proc/meminfo

In [ ]:
!pip install -q keras
!pip install -q Pillow

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()


''

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
#https://github.com/astrada/google-drive-ocamlfuse/
# !mkdir -p drive
# !google-drive-ocamlfuse drive
# !ls

In [ ]:
!ls drive

MyDrive


In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras. layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import glob
import numpy as np
import os
import shutil
import cv2 as cv
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"]=(18,8)
%matplotlib inline
np.random.seed(42)

# Configuring İmage datasets folder 

In [ ]:
dataset_dir=f"/content/drive/My Drive/spectrograms/"


In [ ]:
for root, dirs, files in os.walk(dataset_dir, topdown=False):
    print(root)
    print(f"files type :{type(files)}")
    print(f"files lenth :{len(files)}")
    #print(os.path.join(root, name))
    print(f"DİR type :{type(dirs)}")
    print(f"DİR lenth :{len(dirs)}")
    
    

In [ ]:
for root,dirname, filenames in os.walk(dataset_dir,):
    for filename in filenames:
        print(os.path.join(root, filename))

In [ ]:
!ls

drive  sample_data


# Data Preliminary Exploration

In [ ]:
# create image datasets 

In [ ]:
#  IMG_DIM = (150, 150)

train_files = glob.glob(str(os.path.join(dataset_dir,"1"))+'/*')


print(train_files)
print(f"1 spectogram img lenth :{len(train_files)}")
train_imgs = [img_to_array(load_img(img)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = np.array([fn.split('/')[5].split(".")[0].strip() for fn in train_files])


print('Train dataset shape:', train_imgs.shape)
print('Train labels shape:', train_labels.shape)

[]
1 spectogram img lenth :0
Train dataset shape: (0,)
Train labels shape: (0,)


In [ ]:
train_labels[428]

IndexError: ignored

In [ ]:
image_size=(374,500)
batch_size=32

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1007,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
train_ds.class_names

In [ ]:
#Checking images and labels shapes (amount of images, height, width, color channels)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1007,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
#Checking images and labels shapes (amount of images, height, width, color channels)
for image_batch, labels_batch in test_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Veriden örnekleri görselleştirme

In [ ]:
#Displaying image samples 
plt.figure(figsize=(18, 18))
for images, labels in train_ds.take(1):
    for i in range(batch_size):
        ax = plt.subplot(4,8,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

# Data Preparation

In [ ]:
import os, sys, stat
#Defining parameters for the loader:
batch_size = 32
img_height = image_size[0]
img_width = image_size[1]

#Filtering out corrupted images
num_skipped = 0
for folder_name in (train_labels):
    folder_path = os.path.join(dataset_dir, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()
        if not is_jfif:
            num_skipped += 1
            corrupted_dist="../content/sample_data/corrupted_img/"
            if not os.path.exists(corrupted_dist):
             os.mkdir(corrupted_dist)

            shutil.copy(fpath,corrupted_dist)
            # Delete corrupted image
            # os.chmod(fpath,stat.S_IXUSR)
            # os.remove(fpath)
            
print("Deleted %d images" % num_skipped)

#Data augmentation
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1)])

#Setting train/test split
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=(img_height, img_width),
    batch_size=batch_size)
ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=(img_height, img_width),
    batch_size=batch_size)



# Data Exploration

In [ ]:
#Displaying a random augmented image sample
plt.figure(figsize=(10, 10))
for images, _ in ds_test.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

# Modelling

In [ ]:
model = Sequential() 

**Katlanların oluşturulması**

In [ ]:
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

 
model.add(Conv2D(64, (3, 3), activation='relu'))


model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.25))


model.add(Flatten())


model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))


model.add(Dense(num_classes, activation='softmax'))

Modell Görselleştirme


In [ ]:
model.summary()


In [ ]:

model.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics = ['accuracy'])

### 

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
model.save('save_models/mnist_model.h5')

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])


**Rastgele değer için test işlemi**

In [ ]:
model_test = model.save('save_models/mnist_model.h5')

In [ ]:
test_image = x_test[32]
y_test[32]

In [ ]:
plt.imshow(test_image.reshape(28,28))

In [ ]:
test_data = x_test[32].reshape(1,28,28,1)
pre=model_test.predict(test_data, batch_size=1)

# Model Deployment